# Atividade 5 - Processamentos de Dados (Spark)

Discente: Tiago César da Silva Lopes

Data: 21/09/23

## Link
https://www.who.int/data/gho/data/indicators/indicator-details/GHO/life-expectancy-at-birth-(years)


## Orientações

O dataset informa a expectativa de vida no nascimento e na idade de 60 anos em vários países. Note que quanto maior for o valor da expectativa de vida, melhor deve ser as condições de vida em um país.

## Instruções
Considerando a base de dados definida pelo aluno na Atividade 3, realize atividades de processamento, iniciada em sala (limpeza, seleção...) usando spark.

Pode usar o material compartilhado como exemplo, no entanto, deve ser aplicado a base de dados escolhida pelo aluno (individual).

## Instalação e exibição inicial do dataset

In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.1-py2.py3-none-any.whl size=311285387 sha256=080a08b9fa9fb7a1cca3f5729fb9fee306e0c55d538e476fb89d5f533d9d1bd4
  Stored in directory: /root/.cache/pip/wheels/0d/77/a3/ff2f74cc9ab41f8f594dabf0579c2a7c6de920d584206e0834
Successfully built pyspark


Iniciando uma sessão local e importando os dados:


In [2]:
# Iniciar uma sessão local e importar dados
from pyspark.sql import SparkSession
sc = SparkSession.builder.master('local[*]').getOrCreate()

In [3]:
# carregar dados do meu computador
dados_spark = sc.read.csv("Life expectancy at birth (years).csv", header=True)

# Verificando o tipo de objeto criado
type(dados_spark)

pyspark.sql.dataframe.DataFrame

In [44]:
# Verificando o dataset
dados_spark.show(5)

+-------------+--------------------+---------+------------------+--------------+-------------+-------------------+--------------------+-----------+------+------------+---------+----------+-------------+---------+----+-------------+---------+----+-------------+----------------------+----------+----------------------+----------------+------------+-------------------------+-------------------+--------------------------+--------------------+-----+----------------------+------------+--------+--------------------+
|IndicatorCode|           Indicator|ValueType|ParentLocationCode|ParentLocation|Location type|SpatialDimValueCode|            Location|Period type|Period|IsLatestYear|Dim1 type|      Dim1|Dim1ValueCode|Dim2 type|Dim2|Dim2ValueCode|Dim3 type|Dim3|Dim3ValueCode|DataSourceDimValueCode|DataSource|FactValueNumericPrefix|FactValueNumeric|FactValueUoM|FactValueNumericLowPrefix|FactValueNumericLow|FactValueNumericHighPrefix|FactValueNumericHigh|Value|FactValueTranslationID|FactComments|Lan

In [5]:
# Verificando o schema() deste spark dataframe
dados_spark.printSchema()

root
 |-- IndicatorCode: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- ValueType: string (nullable = true)
 |-- ParentLocationCode: string (nullable = true)
 |-- ParentLocation: string (nullable = true)
 |-- Location type: string (nullable = true)
 |-- SpatialDimValueCode: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Period type: string (nullable = true)
 |-- Period: string (nullable = true)
 |-- IsLatestYear: string (nullable = true)
 |-- Dim1 type: string (nullable = true)
 |-- Dim1: string (nullable = true)
 |-- Dim1ValueCode: string (nullable = true)
 |-- Dim2 type: string (nullable = true)
 |-- Dim2: string (nullable = true)
 |-- Dim2ValueCode: string (nullable = true)
 |-- Dim3 type: string (nullable = true)
 |-- Dim3: string (nullable = true)
 |-- Dim3ValueCode: string (nullable = true)
 |-- DataSourceDimValueCode: string (nullable = true)
 |-- DataSource: string (nullable = true)
 |-- FactValueNumericPrefix: string (nullable = t

In [6]:
#Retornar o número de linhas
dados_spark.count()

4392

## Criando um novo dataset com colunas selecionadas

O dataset original realmente precisa ser limpo e tratado. Dito isso, vamos gerar um novo dataset.

In [42]:
#Selecionar colunas
dados_selecionados = dados_spark.select("Indicator","ParentLocation","Location","Period","Dim1","Value").show(truncate=False)

+--------------------------------+---------------------+------------------------+------+----------+-----+
|Indicator                       |ParentLocation       |Location                |Period|Dim1      |Value|
+--------------------------------+---------------------+------------------------+------+----------+-----+
|Life expectancy at birth (years)|Africa               |Lesotho                 |2019  |Male      |47.66|
|Life expectancy at birth (years)|Africa               |Central African Republic|2019  |Male      |50.21|
|Life expectancy at birth (years)|Africa               |Lesotho                 |2019  |Both sexes|50.75|
|Life expectancy at birth (years)|Africa               |Central African Republic|2019  |Both sexes|53.1 |
|Life expectancy at birth (years)|Africa               |Eswatini                |2019  |Male      |53.36|
|Life expectancy at birth (years)|Eastern Mediterranean|Somalia                 |2019  |Male      |54.01|
|Life expectancy at birth (years)|Africa      

In [14]:
#Principais estatísticas
dados_spark.describe('Period').show(5)

+-------+-----------------+
|summary|           Period|
+-------+-----------------+
|  count|             4392|
|   mean|           2011.0|
| stddev|7.107144348969578|
|    min|             2000|
|    max|             2019|
+-------+-----------------+



In [12]:
dados_spark.describe('Value').show(5)

+-------+------------------+
|summary|             Value|
+-------+------------------+
|  count|              4392|
|   mean| 44.66155965391605|
| stddev|26.433431379026093|
|    min|             10.15|
|    max|             86.94|
+-------+------------------+



In [23]:
# Importar a função "col" para trabalhar com colunas
from pyspark.sql.functions import col

# Contar as ocorrências únicas e ordenar os resultados
dados_spark.groupBy("Indicator").count().orderBy(col("count").desc()).show(truncate=False)

+---------------------------------+-----+
|Indicator                        |count|
+---------------------------------+-----+
|Life expectancy at age 60 (years)|2196 |
|Life expectancy at birth (years) |2196 |
+---------------------------------+-----+



In [39]:
#Ocorrências em uma variável
dados_spark.select("Indicator").distinct().show(truncate=False)

+---------------------------------+
|Indicator                        |
+---------------------------------+
|Life expectancy at age 60 (years)|
|Life expectancy at birth (years) |
+---------------------------------+



In [40]:
#importar sql: groupBy
from pyspark.sql import functions as F

dados_spark.groupBy("Indicator").agg(F.sum("Value")).show(truncate=False)

+---------------------------------+------------------+
|Indicator                        |sum(Value)        |
+---------------------------------+------------------+
|Life expectancy at age 60 (years)|41949.12999999992 |
|Life expectancy at birth (years) |154204.43999999986|
+---------------------------------+------------------+



## Análise e extração de informações

### Qual o país com maior expectativa de vida?

In [38]:
pais_maiorExpectativa2 = dados_spark.groupBy('ParentLocation').agg(F.count('Value').alias('count(Value)')).orderBy(F.desc('count(Value)')).show(truncate=False)

+---------------------+------------+
|ParentLocation       |count(Value)|
+---------------------+------------+
|Europe               |1200        |
|Africa               |1128        |
|Americas             |792         |
|Western Pacific      |504         |
|Eastern Mediterranean|504         |
|South-East Asia      |264         |
+---------------------+------------+



### Qual é maior: De acordo com os dados, a expectativa de vida no nascimento ou quando se tem 60 anos?

In [36]:
maior_expectativaVida = dados_spark.groupBy('Indicator').agg(F.sum('Value')).orderBy(F.sum('Value').desc()).show(truncate=False)

+---------------------------------+------------------+
|Indicator                        |sum(Value)        |
+---------------------------------+------------------+
|Life expectancy at birth (years) |154204.43999999986|
|Life expectancy at age 60 (years)|41949.12999999992 |
+---------------------------------+------------------+



Logo, podemos concluir que a expectativa de vida nos países do mundo é maior no nascimento (154204.44) do que na população com 60 anos (41949.13).

154204.44 > 41949.13

### Qual é o gênero com maior expectativa de vida?

In [47]:
genero_MaiorExpectativa = dados_spark.groupBy('Dim1').agg(F.sum('Value')).orderBy(F.sum('Value').desc()).show()

+----------+------------------+
|      Dim1|        sum(Value)|
+----------+------------------+
|    Female| 68226.84000000007|
|Both sexes|  65398.9799999999|
|      Male|62527.749999999876|
+----------+------------------+



Para a surpresa de muitas pessoas, inclusive a minha, a expectativa de vida é maior para o gênero Feminino. De acordo com o dataset, a média de ambos os sexos é 65,398.98, enquanto que o feminino é 68,226.84 e o masculino é 62,527.75.

Logo, podemos concluir que as mulheres vivem mais do que os homens, mas isto eu deixo aberto para mais investigações.

 68,226.84 > 62,527.75

### Qual é o período que teve a maior expectativa de vida?

In [48]:
periodo_maiorExpectativa = dados_spark.groupBy('Period').agg(F.sum('Value')).orderBy(F.sum('Value').desc()).show()

+------+-----------------+
|Period|       sum(Value)|
+------+-----------------+
|  2019|         50730.63|
|  2015|49972.70000000002|
|  2010|48899.16999999988|
|  2000|46551.06999999991|
+------+-----------------+



O ano com maior taxa de expectativa de vida foi o mais recente, 2019. Logo, podemos concluir que conforme os anos vão passando e as tecnologias evoluindo, a expectativa de vida também vai melhorando pelo mundo.

### Qual é a localização com maior expectativa de vida?

In [51]:
location_maiorExpectativa = dados_spark.groupBy('Location').agg(F.sum('Value')).orderBy(F.sum('Value').desc()).show()

+-----------------+------------------+
|         Location|        sum(Value)|
+-----------------+------------------+
|            Japan|1298.2999999999997|
|      Switzerland|           1276.26|
|        Australia|1275.9399999999998|
|            Spain|1269.6999999999998|
|            Italy|           1266.95|
|        Singapore|           1266.51|
|           France|1266.2700000000002|
|          Iceland|           1263.99|
|           Canada|           1262.56|
|           Israel|           1258.79|
|           Sweden|1258.2599999999998|
|           Cyprus|1257.7999999999997|
|      New Zealand|1254.7900000000002|
|           Norway|1254.3100000000002|
|       Luxembourg|           1252.72|
|Republic of Korea|           1249.21|
|       Costa Rica|1246.4100000000005|
|           Kuwait|1245.6799999999998|
|            Malta|1242.6500000000003|
|      Netherlands|1241.2400000000002|
+-----------------+------------------+
only showing top 20 rows



A lista acima exibe os locais com maiores expectativas de vida. Podemos notar nome de vários Locais conhecidos, como Japão, Canadá, Austrália, Espanha, Itália, Costa Rica entre outros. Note que neste dataset, o Japão registrou a maior taxa de expectativa de vida mundial.

### Qual a localização com a menor expectativa de vida?

In [52]:
location_menorExpectativa = dados_spark.groupBy('Location').agg(F.sum('Value')).orderBy(F.sum('Value').asc()).show()

+--------------------+-----------------+
|            Location|       sum(Value)|
+--------------------+-----------------+
|             Lesotho|739.8800000000001|
|Central African R...|           740.28|
|            Eswatini|791.0100000000001|
|             Somalia|807.5100000000001|
|               Haiti|827.0699999999999|
|            Zimbabwe|827.1300000000001|
|          Mozambique|837.7700000000001|
|       Guinea-Bissau|847.6600000000001|
|            Kiribati|852.4899999999998|
|              Zambia|           854.84|
|        Sierra Leone|856.2800000000001|
|            Botswana|           861.79|
|                Chad|862.6500000000001|
|             Burundi|           868.07|
|              Malawi|882.1900000000002|
|       Côte d’Ivoire|885.2099999999999|
|            Cameroon|885.8800000000001|
|              Guinea|886.2400000000001|
|Democratic Republ...|           888.06|
|              Angola|889.3599999999999|
+--------------------+-----------------+
only showing top

Acima é exibido os locais com as menores taxas de expectativas de vida, como o central da África, Haiti, Angola dentre outros. Isso tem fundamentação nos acontecimentos históricos e políticos que assolam essas localizações.

## Valores nulos

In [53]:
#Dados faltantes
dados_spark.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in dados_spark.columns]).show()

+-------------+---------+---------+------------------+--------------+-------------+-------------------+--------+-----------+------+------------+---------+----+-------------+---------+----+-------------+---------+----+-------------+----------------------+----------+----------------------+----------------+------------+-------------------------+-------------------+--------------------------+--------------------+-----+----------------------+------------+--------+------------+
|IndicatorCode|Indicator|ValueType|ParentLocationCode|ParentLocation|Location type|SpatialDimValueCode|Location|Period type|Period|IsLatestYear|Dim1 type|Dim1|Dim1ValueCode|Dim2 type|Dim2|Dim2ValueCode|Dim3 type|Dim3|Dim3ValueCode|DataSourceDimValueCode|DataSource|FactValueNumericPrefix|FactValueNumeric|FactValueUoM|FactValueNumericLowPrefix|FactValueNumericLow|FactValueNumericHighPrefix|FactValueNumericHigh|Value|FactValueTranslationID|FactComments|Language|DateModified|
+-------------+---------+---------+-----------

In [54]:
#substituir valores ausentes
dados_spark_sem_na = dados_spark.fillna({'Dim2 type':'Sem Resposta', 'Dim2':'Sem Resposta' , 'Dim2ValueCode':'Sem Resposta' , 'Dim3 type':'Sem Resposta' , 'Dim3':'Sem Resposta'
, 'Dim3ValueCode' : 'Sem Resposta' , 'DataSourceDimValueCode' : 'Sem Resposta' , 'DataSource' : 'Sem Resposta' , 'FactValueNumericPrefix' : 'Sem Resposta',
 'FactValueUoM' : 'Sem Resposta' , 'FactValueNumericLowPrefix' : 'Sem Resposta' , 'FactValueNumericLow' : 'Sem Resposta' , 'FactValueNumericHighPrefix' : 'Sem Resposta',
'FactValueTranslationID' : 'Sem Resposta' , 'FactComments' : 'Sem Resposta' , 'FactValueNumericHigh' : 'Sem Resposta'                                         })

In [55]:
dados_spark_sem_na.select([F.count(F.when(F.isnull(c), c)).alias(c) for c in dados_spark_sem_na.columns]).show()

+-------------+---------+---------+------------------+--------------+-------------+-------------------+--------+-----------+------+------------+---------+----+-------------+---------+----+-------------+---------+----+-------------+----------------------+----------+----------------------+----------------+------------+-------------------------+-------------------+--------------------------+--------------------+-----+----------------------+------------+--------+------------+
|IndicatorCode|Indicator|ValueType|ParentLocationCode|ParentLocation|Location type|SpatialDimValueCode|Location|Period type|Period|IsLatestYear|Dim1 type|Dim1|Dim1ValueCode|Dim2 type|Dim2|Dim2ValueCode|Dim3 type|Dim3|Dim3ValueCode|DataSourceDimValueCode|DataSource|FactValueNumericPrefix|FactValueNumeric|FactValueUoM|FactValueNumericLowPrefix|FactValueNumericLow|FactValueNumericHighPrefix|FactValueNumericHigh|Value|FactValueTranslationID|FactComments|Language|DateModified|
+-------------+---------+---------+-----------

Os valores nulos foram substituídos pelo texto 'Sem resposta', e assim não tem mais valor nulo no dataset.

In [56]:
# Verificando o novo dataset
dados_spark_sem_na.show(5)

+-------------+--------------------+---------+------------------+--------------+-------------+-------------------+--------------------+-----------+------+------------+---------+----------+-------------+------------+------------+-------------+------------+------------+-------------+----------------------+------------+----------------------+----------------+------------+-------------------------+-------------------+--------------------------+--------------------+-----+----------------------+------------+--------+--------------------+
|IndicatorCode|           Indicator|ValueType|ParentLocationCode|ParentLocation|Location type|SpatialDimValueCode|            Location|Period type|Period|IsLatestYear|Dim1 type|      Dim1|Dim1ValueCode|   Dim2 type|        Dim2|Dim2ValueCode|   Dim3 type|        Dim3|Dim3ValueCode|DataSourceDimValueCode|  DataSource|FactValueNumericPrefix|FactValueNumeric|FactValueUoM|FactValueNumericLowPrefix|FactValueNumericLow|FactValueNumericHighPrefix|FactValueNumericHi

## Exclusão de colunas da base de dados

In [57]:
# Lista das colunas que vão ser excluídas
colunas_a_excluir = [
    'Dim2 type', 'Dim2', 'Dim2ValueCode',
    'Dim3 type', 'Dim3', 'Dim3ValueCode',
    'DataSourceDimValueCode', 'DataSource',
    'FactValueNumericPrefix', 'FactValueUoM',
    'FactValueNumericLowPrefix', 'FactValueNumericLow',
    'FactValueNumericHighPrefix', 'FactValueNumericHigh',
    'FactValueTranslationID', 'FactComments'
]

In [58]:
#Excluir colunas da base de dados
dados_spark = dados_spark.drop(*colunas_a_excluir)
dados_spark.show(5)

+-------------+--------------------+---------+------------------+--------------+-------------+-------------------+--------------------+-----------+------+------------+---------+----------+-------------+----------------+-----+--------+--------------------+
|IndicatorCode|           Indicator|ValueType|ParentLocationCode|ParentLocation|Location type|SpatialDimValueCode|            Location|Period type|Period|IsLatestYear|Dim1 type|      Dim1|Dim1ValueCode|FactValueNumeric|Value|Language|        DateModified|
+-------------+--------------------+---------+------------------+--------------+-------------+-------------------+--------------------+-----------+------+------------+---------+----------+-------------+----------------+-----+--------+--------------------+
|WHOSIS_000001|Life expectancy a...|     text|               AFR|        Africa|      Country|                LSO|             Lesotho|       Year|  2019|        true|      Sex|      Male|          MLE|           47.66|47.66|      E

## Salvando o resultado e convertendo para pandas

In [59]:
# Salvar resultado
dados_spark_sem_na.write.csv("/content/dadosSpark")

In [60]:
# Spark para Pandas
dados_spark_sem_na_pd = dados_spark_sem_na.toPandas()

# Salvar resultado
dados_spark_sem_na_pd.to_csv("dados_pandas.csv")

In [61]:
dados_spark_sem_na_pd.head()

,IndicatorCode,Indicator,ValueType,ParentLocationCode,ParentLocation,Location type,SpatialDimValueCode,Location,Period type,Period,...,FactValueUoM,FactValueNumericLowPrefix,FactValueNumericLow,FactValueNumericHighPrefix,FactValueNumericHigh,Value,FactValueTranslationID,FactComments,Language,DateModified
0,WHOSIS_000001,Life expectancy at birth (years),text,AFR,Africa,Country,LSO,Lesotho,Year,2019,...,Sem Resposta,Sem Resposta,Sem Resposta,Sem Resposta,Sem Resposta,47.66,Sem Resposta,Sem Resposta,EN,2020-12-04T03:00:00.000Z
1,WHOSIS_000001,Life expectancy at birth (years),text,AFR,Africa,Country,CAF,Central African Republic,Year,2019,...,Sem Resposta,Sem Resposta,Sem Resposta,Sem Resposta,Sem Resposta,50.21,Sem Resposta,Sem Resposta,EN,2020-12-04T03:00:00.000Z
2,WHOSIS_000001,Life expectancy at birth (years),text,AFR,Africa,Country,LSO,Lesotho,Year,2019,...,Sem Resposta,Sem Resposta,Sem Resposta,Sem Resposta,Sem Resposta,50.75,Sem Resposta,Sem Resposta,EN,2020-12-04T03:00:00.000Z
3,WHOSIS_000001,Life expectancy at birth (years),text,AFR,Africa,Country,CAF,Central African Republic,Year,2019,...,Sem Resposta,Sem Resposta,Sem Resposta,Sem Resposta,Sem Resposta,53.1,Sem Resposta,Sem Resposta,EN,2020-12-04T03:00:00.000Z
4,WHOSIS_000001,Life expectancy at birth (years),text,AFR,Africa,Country,SWZ,Eswatini,Year,2019,...,Sem Resposta,Sem Resposta,Sem Resposta,Sem Resposta,Sem Resposta,53.36,Sem Resposta,Sem Resposta,EN,2020-12-04T03:00:00.000Z
